# Initial Repetition Recommender system
### Data preperation
First I'll load necessary data and find which ccid's were repeated.
I'm ignoring video_id for now since I need more exploration of it (e.g. how it is distributed).
Though using video_id might contextualise better repetition need, depending on how it was used.

Firstly the itemset will only be the videos which have been repeated.
I will then expand the item set with every video ccid which has been watched.

In [1]:
from constants import data_path, entities_path, relations_path, Path
import pandas as pd

In [2]:
sample_data = Path("mooc_cube_x-100k")

In [3]:
video_id2ccid = pd.read_csv(relations_path / "video_id-ccid.txt", sep="\t", names=["video_id", "ccid"])

In [ ]:
# If using downsampled dataset, i.e. 100k users
#user2video = pd.read_parquet(sample_data / "user-video.parquet.gzip")
usr_vid_exploded = user2video.explode("seq")
user_video_ccid = usr_vid_exploded.assign(video_id=usr_vid_exploded["seq"].str["video_id"]).merge(video_id2ccid).sort_values("user_id")

In [4]:
# Simplifying the full dataset as it is too large
try:
    user2video = pd.read_parquet(relations_path / "user-video-simple.parquet.gzip")
except:
    user_video_simple = []
    with pd.read_json(relations_path / "user-video.json", lines=True, chunksize=10000) as reader:
        for idx, chunk_df in enumerate(reader):
            print("Chunk ", idx+1)
            chunk_exploded = chunk_df.explode("seq")
            reformated_chunk = chunk_exploded.assign(video_id=chunk_exploded["seq"].str["video_id"])[["video_id", "user_id"]]
            #print(reformated_chunk)
            user_video_simple.append(reformated_chunk)
            #if idx == 1: break
    user2video = pd.concat(user_video_simple)
    user2video.to_parquet(relations_path / "user-video-simple.parquet.gzip", compression="gzip")
user2video

,video_id,user_id
0,V_1395633,U_112
0,V_1395635,U_112
0,V_1395636,U_112
0,V_1395639,U_112
0,V_6210799,U_112
...,...,...
310355,V_8156324,U_36053916
310356,V_6268956,U_36053934
310357,V_6388141,U_36053946
310358,V_8244595,U_36053952


In [5]:
user2ccid = user2video.merge(video_id2ccid, on="video_id")
user2ccid

,video_id,user_id,ccid
0,V_1395633,U_112,98742EFD5E62CAF59C33DC5901307461
1,V_1395633,U_1193,98742EFD5E62CAF59C33DC5901307461
2,V_1395633,U_1547,98742EFD5E62CAF59C33DC5901307461
3,V_1395633,U_4081,98742EFD5E62CAF59C33DC5901307461
4,V_1395633,U_4566,98742EFD5E62CAF59C33DC5901307461
...,...,...,...
21292812,V_6303538,U_36051859,0E81EA9BB1C721449C33DC5901307461
21292813,V_6303542,U_36051859,06434196BD73B0479C33DC5901307461
21292814,V_6280889,U_36053097,F950C5F21B9F115A9C33DC5901307461
21292815,V_4674939,U_36053646,FC9FA22360BF33849C33DC5901307461


Generating the table showing how many times a user has re-watched a given video, 
potentially across courses.

In [6]:
user_video_count = user2ccid.value_counts(["ccid", "user_id"]).reset_index().rename(columns={0: "num_watched"})
user_video_count

,ccid,user_id,num_watched
0,B4A948499263D5AF9C33DC5901307461,U_19828107,6535
1,47A8B4252B8FB6DD9C33DC5901307461,U_19828107,6534
2,CB5055977A5DBFD09C33DC5901307461,U_34007795,2650
3,B482F967992FEDE79C33DC5901307461,U_32139860,2530
4,B4E3E8ADDB1CE7B59C33DC5901307461,U_12135324,2525
...,...,...,...
1978968,582AED0FD055E3EF9C33DC5901307461,U_33769832,1
1978969,582AED0FD055E3EF9C33DC5901307461,U_33769586,1
1978970,582AED0FD055E3EF9C33DC5901307461,U_33768094,1
1978971,582AED0FD055E3EF9C33DC5901307461,U_33767892,1


In [103]:
user2ccid[user2ccid["ccid"] == "98742EFD5E62CAF59C33DC5901307461"].groupby("video_id").nunique()

,user_id,ccid
video_id,,
V_1395633,45234,1
V_4552849,26,1
V_5412731,1,1
V_6020712,45,1
V_608691,1,1
V_6185227,33,1
V_7676262,3,1
V_7915668,9,1
V_8064305,5,1


### Baseline: Implicit feedback
Testing out baseline models for repetition, only considering repeated/not


In [7]:
user_video_reconsumed = (user_video_count[user_video_count["num_watched"] > 1]
                    .rename(columns={"user_id": "user", "ccid": "item", "num_watched": "rating"}))
user_video_reconsumed

,item,user,rating
0,B4A948499263D5AF9C33DC5901307461,U_19828107,6535
1,47A8B4252B8FB6DD9C33DC5901307461,U_19828107,6534
2,CB5055977A5DBFD09C33DC5901307461,U_34007795,2650
3,B482F967992FEDE79C33DC5901307461,U_32139860,2530
4,B4E3E8ADDB1CE7B59C33DC5901307461,U_12135324,2525
...,...,...,...
188519,CF9999A5EA1050339C33DC5901307461,U_28742268,2
188520,1555F0E7D894E1859C33DC5901307461,U_24003099,2
188521,3F36D239097AB4AA9C33DC5901307461,U_31063664,2
188522,F25B858C41E336079C33DC5901307461,U_11631046,2


In [8]:
# Filter out users who have only repeated 1 video - need train and test for each user
user_video_reconsumed_user_count = user_video_reconsumed.groupby("user")["rating"].count().reset_index()
reconsumed_at_least_two_videos = user_video_reconsumed_user_count[user_video_reconsumed_user_count["rating"] > 1]
user_video_reconsumed_filtered = user_video_reconsumed[user_video_reconsumed["user"].isin(reconsumed_at_least_two_videos["user"])]
user_video_reconsumed_filtered

,item,user,rating
0,B4A948499263D5AF9C33DC5901307461,U_19828107,6535
1,47A8B4252B8FB6DD9C33DC5901307461,U_19828107,6534
2,CB5055977A5DBFD09C33DC5901307461,U_34007795,2650
3,B482F967992FEDE79C33DC5901307461,U_32139860,2530
4,B4E3E8ADDB1CE7B59C33DC5901307461,U_12135324,2525
...,...,...,...
188519,CF9999A5EA1050339C33DC5901307461,U_28742268,2
188520,1555F0E7D894E1859C33DC5901307461,U_24003099,2
188521,3F36D239097AB4AA9C33DC5901307461,U_31063664,2
188522,F25B858C41E336079C33DC5901307461,U_11631046,2


#### Sparsity of repetition 
For every video that has been rewatched atleast twice, by at least two different people (i.e. minimum # views = 2)

In [9]:
n_items, n_users = user_video_reconsumed_filtered["item"].unique().size, user_video_reconsumed_filtered["user"].unique().size
n_items, n_users
repetition_sparsity = user_video_reconsumed_filtered.shape[0] / (n_items * n_users)
print(f"Number of users: {n_users},\t Number of items: {n_items}")
print(f"Sparsity of filtered repetition matrix: {repetition_sparsity*100:.4f}%")

Number of users: 22631,	 Number of items: 27235
Sparsity of filtered repetition matrix: 0.0281%


In [10]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn, user_knn, als

In [15]:
# Random Hyper parameters - Could be tuned
algo_ii = item_knn.ItemItem(10)
algo_uu = user_knn.UserUser(10)
algo_als = als.ImplicitMF(50)

In [16]:
def eval(aname, algo, train, test):
    """Generates recommendations over one partition of the dataset"""
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender - 100 or Unlimited recommendations
    recs = batch.recommend(fittable, users, 100) 
    # add the algorithm name for analyzability
    recs['algorithm'] = aname
    return recs

In [17]:
all_recs = []
test_data = []
partitioned = xf.partition_users(user_video_reconsumed_filtered, 5, xf.SampleN(1))
for idx, (train, test) in enumerate(partitioned):
    test_data.append(test)
    print("Partition ", idx+1)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ImplicitALS', algo_als, train, test))

Partition  1


Numba is using threading layer omp - consider TBB
found 1 potential runtime problems - see https://boi.st/lkpy-perf


Partition  2
Partition  3
Partition  4
Partition  5


In [18]:
all_recs_df = pd.concat(all_recs, ignore_index=True)
all_recs_df

,item,score,user,rank,algorithm
0,B5C88081481AB6CF9C33DC5901307461,101.388518,U_1008323,1,ItemItem
1,15E2DDE58D93E7599C33DC5901307461,85.750000,U_1008323,2,ItemItem
2,A5BFA0CF35CD14499C33DC5901307461,85.000000,U_1008323,3,ItemItem
3,6A153520F34A7CA59C33DC5901307461,68.464618,U_1008323,4,ItemItem
4,1006357A7537AF019C33DC5901307461,64.161635,U_1008323,5,ItemItem
...,...,...,...,...,...
4210838,A01ED22C428E8CF99C33DC5901307461,0.066192,U_9992493,96,ImplicitALS
4210839,360A1547525A7EFA9C33DC5901307461,0.066134,U_9992493,97,ImplicitALS
4210840,D3602C39A251663B9C33DC5901307461,0.066088,U_9992493,98,ImplicitALS
4210841,E9410B7E88E86E309C33DC5901307461,0.066048,U_9992493,99,ImplicitALS


In [19]:
test_data = pd.concat(test_data, ignore_index=True)
test_data

,item,user,rating
0,3563CEDCC10A18069C33DC5901307461,U_1008323,2
1,ED38769CA84EA8569C33DC5901307461,U_10095559,2
2,52FF04849B34A14D9C33DC5901307461,U_10096427,2
3,93E2E21002998A6E9C33DC5901307461,U_10174786,2
4,193855E7E165B37B9C33DC5901307461,U_10205902,2
...,...,...,...
22626,9FC856ADE0E849F59C33DC5901307461,U_9830273,4
22627,E4F63BBC773C58C39C33DC5901307461,U_9840297,2
22628,EB10A8439847D7F29C33DC5901307461,U_9880578,2
22629,060181A5BFB5CA039C33DC5901307461,U_9929712,2


In [24]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg, k=10)
rla.add_metric(topn.precision, k=10)
rla.add_metric(topn.recall, k=10)
results = rla.compute(all_recs_df, test_data)
results.head()

nrecs      ndcg  precision  recall
algorithm user                                          
ItemItem  U_1008323      71  0.356207        0.1     1.0
          U_10096427    100  0.000000        0.0     0.0
          U_10205902    100  0.000000        0.0     0.0
          U_10284064      5  1.000000        0.1     1.0
          U_10295316      4  0.000000        0.0     0.0

In [25]:
results.groupby('algorithm').mean()#.#ndcg.mean()

,nrecs,ndcg,precision,recall
algorithm,,,,
ImplicitALS,100.000000,0.301691,0.042508,0.425081
ItemItem,78.238730,0.053675,0.008664,0.086644
UserUser,55.483182,0.053025,0.009144,0.091439


So in this case, there were 22,631 students who reconsumed (re-watched) a video at least twice.
Using a hold-1-out method (to maximise the evaluation set), with 100 recommendations.

From the results above, calculating ANDCG@K, AP@10, and AR@10 (across users), Implicit ALS outperforms drastically.
Since the metrics are sampled (100 recommendations), they probably overestimate the true metrics (ref Sampling paper).

No hyperparameter evaluation

#### Including the full dataset, not only the repeated items
This doesn't really make sense as we're already down sampling the number of recommendations to 100

In [40]:
results.to_parquet(Path("results") / "initial_rep_rs_full.parquet.gzip", compression="gzip")

### Baseline: Implicit feedback with confidence
As there is for each video, a number of how many times a user has repeated a video, one could assume that the person will reconsume it again.

As there are too high memory considerations, it must be run with more memory allocated to it.

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

In [16]:
user_video_view  = user_video_reconsumed_filtered.pivot(index="user", columns="item", values="rating")
user_video_view

MemoryError: Unable to allocate 588. MiB for an array with shape (22631, 27235) and data type bool

In [ ]:
tfidf = TfidfTransformer()
confidence_matrix = tfidf.fit_transform(user_video_view)
#user_video_view_coo = coo_matrix(user_video_view.values)